In [1]:
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'  # carpeta donde se encuentran los datasets
else:
    FUENTES_DIR = '../Fuentes/' # carpeta LOCAL donde se encuentran los scripts
    DATOS_DIR   = '../Datos/' # carpeta LOCAL donde se encuentran los datasets

import sys
sys.path.append(FUENTES_DIR)
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chardet
from sklearn import preprocessing
from ClassNeuronaGral import NeuronaGradiente
from ClassRNMulticlase import RNMulticlase
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier


def openFile(nomArch, sep=None):
    file = DATOS_DIR + nomArch
    #-- detectando la codificación de caracteres usada ----
    with open(file, 'rb') as f:
        result = chardet.detect(f.read()) 
    return pd.read_csv(file, encoding=result['encoding'], sep=sep, engine='python') # or readline if the file is large

#### Ejercicio 1

In [2]:
import matplotlib.pyplot as plt

# Matriz de confusión basada en la imagen
conf_matrix = np.array([
    [17, 0, 1, 0, 1],
    [0, 12, 0, 0, 0],
    [0, 0, 12, 0, 0],
    [2, 0, 0, 38, 0],
    [0, 8, 0, 0, 61]
])

cant_ejemplos = np.sum(conf_matrix)
print("Se usaron", cant_ejemplos, "ejemplos para la evaluación.")

cant_clases = conf_matrix.shape[0]
print("El multiperceptrón puede reconocer", cant_clases, "clases.")

accuracy = np.trace(conf_matrix) / cant_ejemplos
print("Exactitud (accuracy): ", accuracy, " \n")

# -----------------   Precisión por clase   -----------------
# un vector para almacenar la precisión de cada clase
precision = np.zeros(conf_matrix.shape[0])

for i in range(0, len(conf_matrix)):
    # Los verdaderos positivos son los elementos en la diagonal
    verdaderos_positivos = conf_matrix[i][i]
    # Para todos los elementos que el modelo predijo como clase i,
    # VP y FP están en la columna i
    todos_los_positivos = sum(conf_matrix.T[i])
    precision[i] = verdaderos_positivos/todos_los_positivos

print("Precisión por clase:", precision, "\n")

# -----------------   Recall por clase   -----------------
# un vector para almacenar el recall de cada clase
recall = np.zeros(conf_matrix.shape[0])

max = -999
clase = -1
# Recall = nro de aciertos de la clase sobre el nro de ejemplos reales de la clase: VP / (VP + FN)
for i in range(0, len(conf_matrix)):
    # Los verdaderos positivos son los elementos en la diagonal
    verdaderos_positivos = conf_matrix[i][i]
    # Para todos los elementos que son realmente de clase i,
    # VP y FN están en la fila i
    todos_los_reales = sum(conf_matrix[i])
    recall[i] = verdaderos_positivos/todos_los_reales
    if recall[i] >= max:
        max = recall[i]
        clase = i + 1 

print("Recall por clase:", recall, "\n")

# -----------------   F1-Score por clase   -----------------
# Mayor a 1 si precision y recall son mayores a 1
# Menor a 1 si precision y recall son menores a 1
f1_score = 2* (precision*recall) /(precision+recall)
print("F1-Score por clase:", f1_score, "\n")

# -----------------   Accuracy   -----------------
accuracy =  0
for i in range(0, len(conf_matrix)):
    accuracy+=conf_matrix[i][i]

print("Accuracy:", accuracy/conf_matrix.sum())



Se usaron 152 ejemplos para la evaluación.
El multiperceptrón puede reconocer 5 clases.
Exactitud (accuracy):  0.9210526315789473  

Precisión por clase: [0.89473684 0.6        0.92307692 1.         0.98387097] 

Recall por clase: [0.89473684 1.         1.         0.95       0.88405797] 

F1-Score por clase: [0.89473684 0.75       0.96       0.97435897 0.93129771] 

Accuracy: 0.9210526315789473


In [3]:
print("La clase con mayor recall es la clase", clase, "con un recall de", max)

La clase con mayor recall es la clase 3 con un recall de 1.0


#### Ejercicio 2

In [4]:
df = openFile("semillas.csv")

# -1 porque la última columna es la clase
cant_entradas = df.shape[1] -1 
cant_salidas = df["Clase"].unique().size

print(f"Cantidad de entradas: {cant_entradas + 1}")
print("Cantidad de salidas:", cant_salidas)

cant_pesos = ((cant_entradas + 1) * 4) + (4 * cant_salidas)
print("Cantidad de pesos:", cant_pesos)

# -------------------   DUDAS   -----------------
# - ¿El bias cuenta como neurona de entrada?
# - Cuando se dice que la cantidad de neuronas en la capa oculta es 4
#  ¿se refiere a 4 neuronas + 1 bias o 4 neuronas sin contar el bias?

Cantidad de entradas: 8
Cantidad de salidas: 3
Cantidad de pesos: 44


In [ ]:
# b) Los valores que inciden en la dirección de cambio del error son:
# - El error cometido en la clasificación (error de predicción)
# ! Los valores anteriores de los pesos
# - El valor de la derivada de la función de activación


In [6]:
# c) Se modificarían (4 + (4 * canti_entradas))

# ----- Preguntar si está bien -----

#### Ejercicio 3

In [88]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


df = openFile("Vinos.csv")

# Separar datos
X = df.drop(columns=["Class"]).to_numpy()
y = df["Class"].to_numpy()

binariazer = preprocessing.LabelBinarizer() # Clase 1: 0, Clase 2: 1, Clase 3: 2
y = binariazer.fit_transform(y)

X, y = shuffle(X, y, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar datos
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Crear y entrenar red neuronal
funH = 'tanh'
funS = 'softmax'
epocas = 1000
alpha = 0.01
ocultas = [2]

modelo = MLPClassifier(verbose=True, hidden_layer_sizes=ocultas, activation=funH, solver='adam', alpha=alpha, max_iter=epocas, batch_size=20)
modelo.fit(X_train, y_train)

# Predicciones con datos de entrenamiento
y_train_pred = modelo.predict(X_train)
score_train = modelo.score(X_train, y_train)

print("Efectividad en entrenamiento:", (100*(binariazer.inverse_transform(y_train_pred) == binariazer.inverse_transform(y_train)).sum()/len(y_train)))
print("R2 score en entrenamiento:", score_train, "\n")

# Predicciones con datos de test

y_pred = modelo.predict(X_test)
score = modelo.score(X_test, y_test)

y_reales = binariazer.inverse_transform(y_test)
y_pre_reales = binariazer.inverse_transform(y_pred)

# Reporte detallado
print("\nReporte de clasificación:")
print(classification_report(y_reales, y_pre_reales))


# -----------------------  DUDAS -----------------------
# - Los resultados cambian mucho entre ejecuciones.
# - ¿En que basarse para elegir la cantidad de neuronas en la capa oculta? 
# - Optimizadores. Lo que entendí: Para pocos casos, usar L-BFGS.
#   Para muchos casos, usar Adam o SGD.
#   ¿Qué es "pocos" y qué es "muchos"?
# - ¿En qué basarse para elegir el las funciones de activación?

Iteration 1, loss = 2.74625733
Iteration 2, loss = 2.70323552
Iteration 3, loss = 2.66208252
Iteration 4, loss = 2.62180327
Iteration 5, loss = 2.57870312
Iteration 6, loss = 2.53476343
Iteration 7, loss = 2.49162538
Iteration 8, loss = 2.44588822
Iteration 9, loss = 2.40152355
Iteration 10, loss = 2.35775657
Iteration 11, loss = 2.31641745
Iteration 12, loss = 2.27322545
Iteration 13, loss = 2.23191635
Iteration 14, loss = 2.18825659
Iteration 15, loss = 2.14991695
Iteration 16, loss = 2.11267525
Iteration 17, loss = 2.07621558
Iteration 18, loss = 2.04236672
Iteration 19, loss = 2.00996747
Iteration 20, loss = 1.98002479
Iteration 21, loss = 1.94919310
Iteration 22, loss = 1.92105220
Iteration 23, loss = 1.89404560
Iteration 24, loss = 1.86840777
Iteration 25, loss = 1.84295856
Iteration 26, loss = 1.82018627
Iteration 27, loss = 1.79895293
Iteration 28, loss = 1.77748741
Iteration 29, loss = 1.75792993
Iteration 30, loss = 1.73907298
Iteration 31, loss = 1.72011522
Iteration 32, los

/home/santi/Documentos/Cuarto/Deep Learning/mi_entorno/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


#### Ejercicio 4

In [ ]:
df = openFile("Balance.csv")

X = df.drop(columns=["Balance"]).to_numpy()
y = df["Balance"].to_numpy()

binariazer = preprocessing.LabelBinarizer()
y = binariazer.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Normalizar datos
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Crear y entrenar red neuronal
funH = 'tanh'
# funS = 'softmax' No hace falta porque MLPClassifier lo aplica automáticamente
epocas = 1000
alpha = 0.01
ocultas = [20, 10, 10]

modelo = MLPClassifier (
    hidden_layer_sizes=ocultas,
    activation=funH,
    solver='lbfgs',
    alpha=alpha,
    max_iter=epocas,
    batch_size=20
)

modelo.fit(X_train, y_train)

# Predicciones con datos de entrenamiento
y_train_pred = modelo.predict(X_train)
score_train = modelo.score(X_train, y_train)

print("Efectividad en entrenamiento:", (100*(binariazer.inverse_transform(y_train_pred) == binariazer.inverse_transform(y_train)).sum()/len(y_train)))
print("R2 score en entrenamiento:", score_train, "\n")

# Predicciones con datos de test
y_pred = modelo.predict(X_test)
score = modelo.score(X_test, y_test)

y_reales = binariazer.inverse_transform(y_test)
y_pre_reales = binariazer.inverse_transform(y_pred)

# Reporte detallado
print("\nReporte de clasificación con los datos de test:")
print(classification_report(y_reales, y_pre_reales))

# -----------------------  DUDAS -----------------------
# - Lo que hice fue probar varias combinaciones capas y neuronas en cada capa hasta encontrar una que ande bien.
#   ¿Hay alguna forma más sistemática de hacerlo?

Efectividad en entrenamiento: 100.0
R2 score en entrenamiento: 1.0 


Reporte de clasificación con los datos de test:
              precision    recall  f1-score   support

           B       1.00      1.00      1.00        11
           L       1.00      1.00      1.00        55
           R       1.00      1.00      1.00        59

    accuracy                           1.00       125
   macro avg       1.00      1.00      1.00       125
weighted avg       1.00      1.00      1.00       125



#### Ejercicio 5

In [9]:
def imprimir_resultados(train_scores, test_scores):
    # Análisis estadístico
    print("\n" + "="*50)
    print("ANÁLISIS ESTADÍSTICO DE 10 EJECUCIONES")
    print("="*50)

    print(f"Accuracy ENTRENAMIENTO:")
    print(f"  Media: {np.mean(train_scores):.4f} ± {np.std(train_scores):.4f}")
    print(f"  Rango: [{np.min(train_scores):.4f}, {np.max(train_scores):.4f}]")

    print(f"\nAccuracy TEST:")
    print(f"  Media: {np.mean(test_scores):.4f} ± {np.std(test_scores):.4f}")
    print(f"  Rango: [{np.min(test_scores):.4f}, {np.max(test_scores):.4f}]")

    # Detectar overfitting
    overfitting = np.array(train_scores) - np.array(test_scores)
    print(f"\nOverfitting promedio: {np.mean(overfitting):.4f}")

    print(f"\nCONCLUSIÓN:")
    print(f"El modelo tiene una accuracy promedio de {np.mean(test_scores):.3f} ± {np.std(test_scores):.3f}")
    print(f"Con una variabilidad de ±{np.std(test_scores)*100:.1f}% entre ejecuciones")

In [10]:
import Modelo

df = openFile("zoo.csv")
animales = df.drop(columns=["animal"])

X = animales.drop(columns=["Clase"]).to_numpy()
y = animales["Clase"].to_numpy()

# One hot encoding para las clases
binariazer = preprocessing.LabelBinarizer()
y = binariazer.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Normalizar datos
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

mini_lote = animales.shape[0] // 10
epocas = 1000
alpha = 0.001
funH = 'relu'
ocultas = [10]

modelo = Modelo.Modelo(
    X_train,
    y_train,
    ocultas=ocultas,
    funH=funH,
    alpha=alpha,
    epocas=epocas,
    batch_size=mini_lote,
    solver='adam',
    random_state=42
)


In [11]:
# Ejecutar 10 veces
train_scores, test_scores = modelo.evaluar_modelo_multiple_veces(X, y, 10)
# imprimir_resultados(train_scores, test_scores)


--- Ejecución 1/10 ---
Train Accuracy: 0.9857
Test Accuracy: 0.9677

--- Ejecución 2/10 ---
Train Accuracy: 0.9429
Test Accuracy: 1.0000

--- Ejecución 3/10 ---
Train Accuracy: 0.9429
Test Accuracy: 1.0000

--- Ejecución 4/10 ---
Train Accuracy: 0.9857
Test Accuracy: 0.9677

--- Ejecución 5/10 ---
Train Accuracy: 0.9429
Test Accuracy: 1.0000

--- Ejecución 6/10 ---
Train Accuracy: 0.9714
Test Accuracy: 0.9355

--- Ejecución 7/10 ---
Train Accuracy: 0.9571
Test Accuracy: 1.0000

--- Ejecución 8/10 ---
Train Accuracy: 0.9571
Test Accuracy: 0.9677

--- Ejecución 9/10 ---
Train Accuracy: 0.9571
Test Accuracy: 1.0000

--- Ejecución 10/10 ---
Train Accuracy: 0.9714
Test Accuracy: 0.9677


In [12]:
params = {
    'ocultas': [10],
    'funH': 'tanh',
    'solver': 'lbfgs'
}
modelo.set_params(**params)
train_scores, test_scores = modelo.evaluar_modelo_multiple_veces(X, y, 10)
imprimir_resultados(train_scores, test_scores)


--- Ejecución 1/10 ---
Train Accuracy: 0.9857
Test Accuracy: 0.9677

--- Ejecución 2/10 ---
Train Accuracy: 0.9714
Test Accuracy: 1.0000

--- Ejecución 3/10 ---
Train Accuracy: 0.9571
Test Accuracy: 1.0000

--- Ejecución 4/10 ---
Train Accuracy: 0.9857
Test Accuracy: 0.9677

--- Ejecución 5/10 ---
Train Accuracy: 0.9571
Test Accuracy: 1.0000

--- Ejecución 6/10 ---
Train Accuracy: 0.9857
Test Accuracy: 0.9677

--- Ejecución 7/10 ---
Train Accuracy: 0.9714
Test Accuracy: 1.0000

--- Ejecución 8/10 ---
Train Accuracy: 0.9714
Test Accuracy: 1.0000

--- Ejecución 9/10 ---
Train Accuracy: 0.9714
Test Accuracy: 1.0000

--- Ejecución 10/10 ---
Train Accuracy: 0.9714
Test Accuracy: 0.9677

ANÁLISIS ESTADÍSTICO DE 10 EJECUCIONES
Accuracy ENTRENAMIENTO:
  Media: 0.9729 ± 0.0100
  Rango: [0.9571, 0.9857]

Accuracy TEST:
  Media: 0.9871 ± 0.0158
  Rango: [0.9677, 1.0000]

Overfitting promedio: -0.0142

CONCLUSIÓN:
El modelo tiene una accuracy promedio de 0.987 ± 0.016
Con una variabilidad de ±1.6

In [13]:
params = {
    'ocultas': [20, 20, 20],
    'funH': 'tanh',
    'solver': 'lbfgs',
    'alpha': 0.001
}
modelo.set_params(**params)
train_scores, test_scores = modelo.evaluar_modelo_multiple_veces(X, y, 10)
imprimir_resultados(train_scores, test_scores)


--- Ejecución 1/10 ---
Train Accuracy: 0.9857
Test Accuracy: 0.9677

--- Ejecución 2/10 ---
Train Accuracy: 0.9714
Test Accuracy: 1.0000

--- Ejecución 3/10 ---
Train Accuracy: 0.9714
Test Accuracy: 1.0000

--- Ejecución 4/10 ---
Train Accuracy: 0.9857
Test Accuracy: 0.9677

--- Ejecución 5/10 ---
Train Accuracy: 0.9714
Test Accuracy: 1.0000

--- Ejecución 6/10 ---
Train Accuracy: 0.9857
Test Accuracy: 0.9677

--- Ejecución 7/10 ---
Train Accuracy: 0.9714
Test Accuracy: 1.0000

--- Ejecución 8/10 ---
Train Accuracy: 0.9714
Test Accuracy: 1.0000

--- Ejecución 9/10 ---
Train Accuracy: 0.9571
Test Accuracy: 1.0000

--- Ejecución 10/10 ---
Train Accuracy: 0.9714
Test Accuracy: 0.9355

ANÁLISIS ESTADÍSTICO DE 10 EJECUCIONES
Accuracy ENTRENAMIENTO:
  Media: 0.9743 ± 0.0086
  Rango: [0.9571, 0.9857]

Accuracy TEST:
  Media: 0.9839 ± 0.0216
  Rango: [0.9355, 1.0000]

Overfitting promedio: -0.0096

CONCLUSIÓN:
El modelo tiene una accuracy promedio de 0.984 ± 0.022
Con una variabilidad de ±2.2

#### Ejercicio 6

In [14]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
image_segmentation = fetch_ucirepo(id=50) 

# data (as pandas dataframes) 
X = image_segmentation.data.features.to_numpy()
y = image_segmentation.data.targets.to_numpy()


In [91]:
ocultas = [10, 10]
mini_lote = X.shape[0] // 10
epocas = 2000
funH = 'tanh'

In [ ]:
accuracies_train = []
accuracies_test = []

for i in range(10):
    print(f"\n--- Ejecución {i+1}/10 ---")

    X, y = shuffle(X, y, random_state=i)
    
    # División aleatoria DIFERENTE en cada ejecución
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3
    )
    
    # Normalizar
    scaler = preprocessing.StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    binarizer = preprocessing.LabelBinarizer()
    y_train = binarizer.fit_transform(y_train)
    y_test = binarizer.transform(y_test)
    
    # Crear modelo con diferente random_state en cada ejecución
    modelo = MLPClassifier(
        hidden_layer_sizes=ocultas,
        activation=funH,
        solver='adam',
        alpha=0.00001,
        max_iter=epocas,
        batch_size=mini_lote,
        random_state=i,
        validation_fraction=0.3,
    )

    # Entrenar
    modelo.fit(X_train, y_train)

    acc_train = modelo.score(X_train, y_train)
    acc_test = modelo.score(X_test, y_test)

    accuracies_train.append(acc_train)
    accuracies_test.append(acc_test)
    
    print(f"Train Accuracy: {acc_train:.4f}")
    print(f"Test Accuracy: {acc_test:.4f}")

    from sklearn.metrics import classification_report
    y_pred = modelo.predict(X_test)
    print(classification_report(binarizer.inverse_transform(y_test), binarizer.inverse_transform(y_pred)))
